<center>

## DiploDatos: Aprendizaje No Supervisado

## Embeddings con el DataSet FIFA21



## 1.Inicialización_del_Entorno

Empezamos importando algunas herramientas para cargar los datos y manipularlos.

In [1]:
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [2]:
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt

In [3]:
from time import time

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MeanShift, estimate_bandwidth

## 2.Preparación_de_los_Datos

Cargamos los datos bajados de la database de [Kaggle](https://www.kaggle.com/stefanoleone992/fifa-21-complete-player-dataset) como `players_21.csv`.

In [4]:
df = pd.read_csv('players_21.csv')

print(f'Dimensiones: {df.shape}')

Dimensiones: (18944, 106)


In [5]:
# Variables Auxiliares.
name = 'short_name'
age = 'age'
country = 'nationality'
club = 'club_name'
league = 'league_name'
position = 'team_position'
overall = 'overall'
potential = 'potential'

description = [name, age, country, club, league, position, overall, potential]

# Variables Adicionales.
player_class = 'Player Class'
position_type = 'Position Type'
tsne_0 = 'TSNE_0'
tsne_1 = 'TSNE_1'
pca_0 = 'PCA_0'
pca_1 = 'PCA_1'
cluster_KMeans = 'KMeans'
cluster_MeanShift = 'MeanShift'

In [6]:
skillGroup = ('attacking',
              'defending',
              'goalkeeping',
              'skill',
              'power',
              'movement',
              'mentality')

skillRating = {}

# Veamos las habilidades con las que estamos trabajando...
for g in skillGroup:
    # Inicializamos el grupo de habilidades.
    skillRating[g] = []
    # Iteramos sobre el conjunto de columnas del DF.
    for c in df.columns[40:100]:
        if c.startswith(g):
            # Almacenamos la habilidad en el grupo.
            skillRating[g].append(c)

In [7]:
# Conservamos a los jugadores buenos.
minOverall = 70
bestDF = df[(df[overall] > minOverall)]

# Marcamos a los mejores jugadores.
topOverall = 85
bestDF[player_class] = bestDF[overall].apply(lambda o: 'TOP' if o > topOverall else 'Average')

In [8]:
# Generalizamos las posiciones de los jugadores en el juego...
goalkeepers = ['GK']
defenders = ['LCB', 'RCB', 'LB', 'RB', 'CB', 'LWB', 'RWB']
midfielders = ['LM', 'RM', 'CM', 'LCM', 'RCM', 'LDM', 'RDM', 'CDM', 'LAM', 'RAM', 'CAM']
attackers = ['ST', 'LS', 'RS', 'LW', 'RW', 'LF', 'RF', 'CF']

def general_position(position):
    if position in attackers:
        return 'Attacker'
    elif position in midfielders:
        return 'Midfielder'
    elif position in defenders:
        return 'Defender'
    elif position in goalkeepers:
        return 'Goalkeeper'
    else:
        return 'Other'

position_order = ['Goalkeeper', 'Defender', 'Midfielder', 'Attacker', 'Other']

bestDF[position_type] = bestDF[position].apply(general_position)

## 3.Visualización_con_Embeddings_tSNE

**t-SNE: t-distributed Stochastic Neighbor Embedding**

*t-SNE* es una herramienta para visualizar datos altamente dimensionales.

Los proyecta en menos dimensiones (2D o 3D) manteniendo la distancia entre ellos.

Consideramos: filas/observaciones $x_i$ y $x_j$ (jugador i y jugador j) en $R^n$.

Comienza convirtiendo distancias entre puntos (observaciones) en probabilidades condicionales.

$d(x_i, x_j) \rightarrow p(x_j | x_i)$,

Y luego va acomodando puntos asociados (en el plano 2D o el espacio 3D) que ajusten las probabilidades condicionales.

$x'_i$ y $x'_j$ en $R^2$  (para el jugador i y jugador j ) $p(x'_j | x'_i) \leftrightarrow d(x'_i, x'_j)$ / $p(x_j | x_i) \approx p(x'_j | x'_i)$

El ajuste consiste en minimizar la divergencia **Kullback-Leibler** entre las probabilidades condicionales establecidas en el espacio de mayor dimensión y las determinadas en el espacio de menor dimensión.

El modelo de distribución de probabilidad es el de *t-Student*.

*t-SNE* tiene una función de costo que no es convexa, por lo cual diferentes inicializaciones pueden dar diferentes resultados.

In [9]:
onlySkills = []
for skills in skillRating.values():
    onlySkills += skills

# Eliminamos atributos nulos.
onlySkills.remove('defending_marking')

# Sólo con desempeños según habilidades (no vacías).
skillsDF = bestDF[onlySkills]

print(f'Dimensión del espacio de los datos: {len(onlySkills)}')

Dimensión del espacio de los datos: 33


Transformaremos el espacio de **Dim 33** en un espacio de **Dim 2** (`n_components = 2`).

In [10]:
time_start = time()

tsne = TSNE(n_components=2, perplexity=25, verbose=1, random_state=123)
X_tsne = tsne.fit_transform(skillsDF)

time_end = time()

print(f't-SNE done! Time elapsed: {time_end - time_start} seconds.')

[t-SNE] Computing 76 nearest neighbors...
[t-SNE] Indexed 4512 samples in 0.019s...
[t-SNE] Computed neighbors for 4512 samples in 1.113s...
[t-SNE] Computed conditional probabilities for sample 1000 / 4512
[t-SNE] Computed conditional probabilities for sample 2000 / 4512
[t-SNE] Computed conditional probabilities for sample 3000 / 4512
[t-SNE] Computed conditional probabilities for sample 4000 / 4512
[t-SNE] Computed conditional probabilities for sample 4512 / 4512
[t-SNE] Mean sigma: 11.934181
[t-SNE] KL divergence after 250 iterations with early exaggeration: 77.314774
[t-SNE] KL divergence after 1000 iterations: 1.826157
t-SNE done! Time elapsed: 36.30756664276123 seconds.


**El resultado (`X_tsne`) se guarda en una arreglo *numpy* de tamaño...**

- Cantidad de filas `# Jugadores Buenos en DF (~ 4500)`.

- Cantidad de columnas `n_components (= 2)`.

In [11]:
print(f'Dimensiones: {X_tsne.shape}')

X_tsne[:5] # Algunos jugadores cualquiera...

Dimensiones: (4512, 2)


array([[ 48.92104  ,  14.606376 ],
       [ 51.384037 ,  11.476695 ],
       [ -7.7444105, -60.51396  ],
       [ 52.64647  ,  11.556143 ],
       [ 48.813892 ,  14.415254 ]], dtype=float32)

**GRAFICAMOS: Scatter Plot**

Las variables que usamos para el *scatterplot* son las 2 componentes dadas por **tSNE**.

Agregamos los nombres de los jugadores top (`overall` mayor a 85), y analizamos una tercera variable recorriendo los puntos sobre el gráfico.

In [12]:
# Diferenciamos a los mejores jugadores.
bool_TOP = bestDF[overall] > topOverall

In [13]:
# Graficamos los datos en el espacio transformado por las componentes TSNE.

graph = go.Scatter(x=X_tsne[:, 0],
                   y=X_tsne[:, 1],
                   name='Players',
                   mode='markers',
                   text=bestDF[position],
                   marker=dict(size=5)
                  )

top = go.Scatter(x=X_tsne[bool_TOP, 0],
                 y=X_tsne[bool_TOP, 1],
                 name='Top Players',
                 text=bestDF.loc[bool_TOP, name],
                 textfont=dict(family='sans serif', size=8, color='black'),
                 opacity=0.9,
                 mode='text'
                )

data = [graph, top]

layout = go.Layout(title='t-SNE - FIFA Players',
                   titlefont=dict(size=20),
                   xaxis=dict(title='Componente 0'),
                   yaxis=dict(title='Componente 1'),
                   autosize=False,
                   width=900,
                   height=900
                  )

fig = go.Figure(data=data, layout=layout)
fig.show()

**INTERPRETACIÓN**

Un poco de análisis visual supervisado usando *tsne*.
A continuación pondremos a prueba la intuición o preconcepto de que los jugadores se agrupan en el espacio de las habilidades (transformado / reducido por *tsne*) según la posición en el campo de juego...

In [14]:
bestDF[position_type].value_counts()\
                     .reset_index()\
                     .rename(columns={'index': position_type, position_type: '#Players'})\
                     .set_index(position_type)

,#Players
Position Type,
Other,1778
Midfielder,1035
Defender,905
Attacker,501
Goalkeeper,293


Recordemos que la posición **Other** comprende a todos los jugadores suplentes y reservas.
Es fundamental notar que estos jugadores también tienen posiciones definidas en sus equipos, por lo que es posible agruparlos observando sólo los valores de sus habilidades.

**GRAFICAMOS** 

Las variables que usamos para el *scatterplot* son las 2 componentes dadas por **tSNE**.

Diferenciamos usando vectores booleanos para cada posición, y si es un jugador *top*.

In [15]:
bool_GK = bestDF[position_type] == 'Goalkeeper'

bool_D = bestDF[position_type] == 'Defender'

bool_M = bestDF[position_type] == 'Midfielder'

bool_A = bestDF[position_type] == 'Attacker'

In [16]:
palette = ['navy', 'red', 'yellow', 'green', 'skyblue']

attacker = go.Scatter(x=X_tsne[bool_A, 0],
                      y=X_tsne[bool_A, 1],
                      name='Attackers',
                      text=bestDF.loc[bool_A, name],
                      opacity=0.8,
                      marker=dict(color=palette[0], size=5),
                      mode='markers'
                     )

midfielder = go.Scatter(x=X_tsne[bool_M, 0],
                        y=X_tsne[bool_M, 1],
                        name='Midfielders',
                        text=bestDF.loc[bool_M, name],
                        opacity=0.7,
                        marker=dict(color=palette[1], size=5),
                        mode='markers'
                       )

defender = go.Scatter(x=X_tsne[bool_D, 0],
                      y=X_tsne[bool_D, 1],
                      name='Defenders',
                      text=bestDF.loc[bool_D, name],
                      opacity=0.7,
                      marker=dict(color=palette[2], size=5),
                      mode='markers'
                     )

goalkeeper = go.Scatter(x=X_tsne[bool_GK, 0],
                        y=X_tsne[bool_GK, 1],
                        name='Goalkeepers',
                        text=bestDF.loc[bool_GK, name],
                        opacity=0.5,
                        marker=dict(color=palette[3], size=5),
                        mode='markers'
                       )

top = go.Scatter(x=X_tsne[bool_TOP, 0],
                 y=X_tsne[bool_TOP, 1],
                 name='Top Players',
                 text=bestDF.loc[bool_TOP, name],
                 textfont=dict(family='sans serif', color='black', size=10),
                 opacity=0.9,
                 mode='text'
                )

data = [attacker, midfielder, defender, goalkeeper, top]

layout = go.Layout(title='t-SNE - FIFA Players',
                   titlefont=dict(size=20),
                   xaxis=dict(title='Componente 0'),
                   yaxis=dict(title='Componente 1'),
                   autosize=False,
                   width=900,
                   height=900
                  )

fig = go.Figure(data=data, layout=layout)
fig.show()

Como conclusión, podemos ver que en este espacio 2D es realmente muy evidente la separación de los arqueros del resto de los jugadores (lo cual es lógico pues sus habilidades son muy diferentes).

El agrupamiento visual del resto de jugadores, en este espacio, no es tan evidente pues los clusters no son disconexos.

Las posiciones no explican tan claramente los clusters encontrados.

Una explicación posible puede ser porque hay jugadores ambivalentes que unen los espacios entre defensa, mediocampo, y ataque.

¿Habrá otra explicación?¿Qué otras variables (que no estamos graficando) pueden explicar los grupos? 

¿Cuántos grupos encontrará **MeanShift**, o cuales **KMeans**?

**Respuesta**

Es una suposición bastante convincente que los jugadores ambivalentes sean los encargados de unir los agrupamientos de posiciones en el campo de los jugadores.
Después de todo, se observa a los defensores en un extremo y a los atacantes en el otro extremo del gráfico.
El grupo que parece hacer de puente de estos conjuntos es el de mediocampistas.

Un defensor puede jugar más adelantado para aportar a la creación, mientras que un delantero puede jugar más atrasado para ayudar en la presión.

A simple vista parece que no existen otros atributos, en el conjunto de datos, que puedan servir para separar a los jugadores del campo según sus posiciones.
Quizás otras variables que podrían ser útiles en la visualización serían las relacionadas al desempeño (o vinculación) del jugador en cada una de las posiciones del juego (las últimas del *dataframe*).
Claramente si adoptáramos este análisis nos estaríamos acercando a un problema supervisado.

# 4.Clustering_tSNE

**KMeans**

Intentamos agrupar sobre los componentes de *tSNE*.
Se define de forma arbitraria la cantidad de conjuntos a buscar.

In [17]:
# Número de clusters buscados.
n_clust = 4

km = KMeans(n_clusters=n_clust, random_state=123)
km.fit(X_tsne) # Utiliza todas las componentes: 2 dimensiones.

# Recuperación de las etiquetas.
clusters = km.labels_

print(f'Suma de los cuadrados de las distancias al cluster (Inertia) {km.inertia_}')

Suma de los cuadrados de las distancias al cluster (Inertia) 2062212.5046455387


In [18]:
clustersDF = bestDF.copy()
clustersDF[cluster_KMeans] = clusters.astype(str) # Clusters como Variables Categóricas
clustersDF[tsne_0] = X_tsne[:, 0]
clustersDF[tsne_1] = X_tsne[:, 1]

print(f'KMeans encontró {max(clusters) + 1} clusters (nosotros forzamos la cantidad)')

KMeans encontró 4 clusters (nosotros forzamos la cantidad)


In [19]:
fig = px.scatter(clustersDF,
                 x=tsne_0,
                 y=tsne_1,
                 color=cluster_KMeans,
                 hover_name=name,
                 hover_data=description,
                 symbol=player_class,
                 symbol_map={'TOP': 'star', 'Average': 'circle'},
                 title='Clustering KMeans',
                 height=800,
                 width=800)

fig.show()

**Observación**

Si intentamos analizar el gráfico exclusivamente de forma visual, **KMeans** obtiene un resultado satisfactorio.
Separa el conjunto de los arqueros a la perfección y agrupa de forma más o menos convincente al resto de jugadores de campo (defensas, mediocampistas, delanteros).

Es importante notar que **KMeans** solo contaba con los atributos obtenidos por *tSNE* (es decir, 2 dimensiones).
Por lo que cualquier imperfección observable en el gráfico es causada por la reduccion de dimensionalidad previa.

**MeanShift**

Intentamos agrupar sobre los componentes de tSNE.
Al estimar el ancho de banda óptimo (`42.94`) para aplicar el agrupamiento, solo se encontraron **2** conjuntos.
Debido que esto no es el resultado que se buscaba, se definió un valor arbitrario para efectuar el *clustering*.

In [20]:
# Estimate BW for MeanShift.
estimated_BW = estimate_bandwidth(X_tsne, random_state=123)
print(f'BandWidth Estimado: {estimated_BW}')

# Utilizamos el ancho de banda encontrado.
ms = MeanShift(bandwidth=estimated_BW, bin_seeding=True)
ms.fit(X_tsne) # Utiliza todas las componentes: 2 dimensiones.

# Recuperación de las etiquetas.
clusters = ms.labels_

print(f'MeanShift encontró {max(clusters) + 1} clusters (según los hiperparámetros elegidos)')

BandWidth Estimado: 42.93550849780112
MeanShift encontró 2 clusters (según los hiperparámetros elegidos)


In [21]:
# Utilizamos un ancho de banda arbitrario.
ms = MeanShift(bandwidth=25, bin_seeding=True)
ms.fit(X_tsne) # Utiliza todas las habilidades: 2 dimensiones.

# Recuperación de las etiquetas.
clusters = ms.labels_

In [22]:
clustersDF[cluster_MeanShift] = clusters.astype(str) # Clusters como Variables Categóricas

print(f'MeanShift encontró {max(clusters) + 1} clusters (según los hiperparámetros elegidos)')

MeanShift encontró 4 clusters (según los hiperparámetros elegidos)


In [23]:
fig = px.scatter(clustersDF,
                 x=tsne_0,
                 y=tsne_1,
                 color=cluster_MeanShift,
                 hover_name=name,
                 hover_data=description,
                 symbol=player_class,
                 symbol_map={'TOP': 'star', 'Average': 'circle'},
                 title='Clustering MeanShift',
                 height=800,
                 width=800)

fig.show()

**Observación**

Los agrupamientos obtenidos son similares a los encontrados por *KMeans*, 
 el grupo de los defensores es más pequeño mientras que el grupo de los atacantes es más grande.
Los conjuntos encontrados parecen bastante buenos y se separan bien.

# 5.Visualización_con_Embeddings_PCA

**PCA**

Para reducción de dimensión, de **n** dimensiones a **m** dimensiones con $m << n$.

**Ejemplo:** Vía Láctea en **3D**, [ejemplo 1](https://youtu.be/WNASVRwdf9Q) o [ejemplo 2](https://www.youtube.com/watch?v=S7GiZMVNB20), a Vía Láctea en **2D** (la proyección en 2D que mejor describe la dispersión de los datos).

**Variables de Desempeño (*skillRating*)**

Retiramos a los arqueros, jugadores suplentes, jugadores reserva, y aquellos jugadores con un bajo `overall`.
Complementamos el procesamiento restringiendo los datos a las variables numéricas que estudiaremos.

In [24]:
# Eliminamos posiciones no interesantes.
pcaDF = bestDF[~bestDF[position_type].isin(['Goalkeeper', 'Other'])]

onlySkills = []
for group, skills in skillRating.items():
    if group != 'goalkeeping':
        onlySkills += skills

onlySkills.remove('defending_marking')

# Sólo con desempeños según habilidades (no vacías).
skillsDF = pcaDF[onlySkills]

print(f'Dimensión del espacio de los datos: {len(onlySkills)}')

Dimensión del espacio de los datos: 28


**Se computan los Componentes Principales**

Cuando se usa *decomposition.PCA*...

La descomposición **PCA** de *sklearn* centra los datos pero no los estandariza (input data is centered but not scaled for each feature before applying the *SVD*).
Por lo tanto, debemos **estandarizarlos**.

In [25]:
scaler = StandardScaler().fit(skillsDF)

# Numpy Array Estandarizado (le resta la media y divide por el desvío).
X_scaled = scaler.transform(skillsDF)

print(f'Sin Estandarización:\n{skillsDF.values[0]}')
print(f'Con Estandarización:\n{X_scaled[0]}')

Sin Estandarización:
[85 95 70 91 88 35 24 96 93 94 91 96 86 68 72 69 94 91 80 91 94 95 44 40
 93 95 75 96]
Con Estandarización:
[ 1.58781115  2.20983863  0.50663851  2.96692943  2.04581892 -1.24896221
 -1.58679086  2.47696231  2.07928338  2.39957698  2.58133557  3.25363598
  1.49720511 -0.11895272 -0.26194958 -0.10224895  2.14581975  1.61353507
  0.76225029  1.66669751  3.71438729  2.0760533  -1.91634089 -1.00220432
  1.9668365   2.43960413  1.20733223  3.48947098]


**DESCOMPOSICIÓN PCA**

En la descomposición podemos elegir `n_components` para quedarnos con una cantidad chica de componentes.
La idea es que sean bastante menos que la dimensión de lo datos pero que describan un buen porcentaje de la dispersión de los mismos.

In [26]:
# Elegimos 5, pero podrían ser más...
pca = PCA(n_components=5)

# Input data is centered but not scaled for each feature before applying the SVD.
pca.fit(X_scaled)

# Proporción de Varianza.
print(f'Proporción de Varianza por componente: {pca.explained_variance_ratio_}')
# Proporción de Varianza Acumulada.
print(f'Proporción de Varianza Acumulada por componente: {pca.explained_variance_ratio_.cumsum()}')

# Numpy Array Proyectado.
X_projected = pca.transform(X_scaled)
print(f'Tamaño del conjunto de datos: {X_projected.shape}')

Proporción de Varianza por componente: [0.41671552 0.14548482 0.11079231 0.06663965 0.03862529]
Proporción de Varianza Acumulada por componente: [0.41671552 0.56220034 0.67299265 0.7396323  0.77825759]
Tamaño del conjunto de datos: (2441, 5)


In [27]:
print(X_projected[:5]) # 5 primeros jugadores (con sus 'n_components').

[[-9.5157544  -2.96546631  1.36049839 -0.69240234  3.06442998]
 [-7.50599741 -2.24306836  3.17656327 -4.17304796  1.74528267]
 [-6.07776798 -2.49398721  3.74251881 -2.97256206  1.11498338]
 [-9.08199913 -1.71675438  0.32273867 -0.73301999  2.71831111]
 [-7.19803502 -5.47883076  0.83207263 -0.39625496  0.84130984]]


In [28]:
pcW = pca.components_ # Composición de las primeras componentes.

print(pcW) # Aporte de las variables skillRating.

[[-0.20292767 -0.24991836  0.12912988 -0.16421329 -0.23775176  0.16747922
   0.17214502 -0.26083644 -0.24072233 -0.20904366 -0.08329056 -0.24541231
  -0.2011293   0.07474732 -0.05275033  0.15238484 -0.24444176 -0.17208566
  -0.13941488 -0.2153121  -0.10606959 -0.19078317  0.13392962  0.16021679
  -0.25436772 -0.2424587  -0.20518684 -0.13271625]
 [-0.11514953  0.06482762 -0.10284538 -0.32707351  0.01128914 -0.34527803
  -0.3269021  -0.03481843 -0.09518063 -0.10305965 -0.36810085 -0.14764683
  -0.08404933 -0.05311661 -0.16556146 -0.10311765 -0.06541643  0.15339898
   0.16121839  0.08682569 -0.25563357  0.02744536 -0.27250197 -0.35399542
   0.01638282 -0.15330118  0.00144866 -0.24873651]
 [-0.20673092  0.19077885  0.32467749 -0.01936284  0.20870662 -0.1814757
  -0.20528033 -0.07155398 -0.01758201  0.05415345 -0.09957057  0.00524093
   0.21881949  0.04374953 -0.21208281  0.30673804  0.13505497 -0.30138718
  -0.26177563 -0.27475846  0.11824026 -0.29733939  0.05311356 -0.17043416
   0.085074

El siguiente gráfico muestra la composición de los pesos de las **dos** primeras componentes principales.

In [29]:
data = []

for i, (x, y) in enumerate(zip(pcW[0, :], pcW[1, :])):
    graph = go.Scatter(x=[0, x],
                       y=[0, y],
                       text=skillsDF.columns[i],
                       mode='lines+markers+text',
                       textposition='top left',
                       textfont=dict(family='sans serif', size=15)
                      )
    data.append(graph)

layout = go.Layout(title='PCA - FIFA Skills',
                   titlefont=dict(size=20),
                   xaxis=dict(title='Componente 0'),
                   yaxis=dict(title='Componente 1'),
                   autosize=False,
                   width=900,
                   height=900,
                   showlegend=False
                  )

fig = go.Figure(data=data, layout=layout)
fig.show()

**Observación**

Notar que las habilidades forman una especie de *medialuna* en el plano.

En sentido antihorario...

- 1º cuadrante no aparecen habilidades, lo cual significa que los jugadores con habilidades altas **NO** aparecerán aquí. Debido que filtramos a los jugadores buenos, posiblemente este cuadrante esté bastante despoblado.
- 2º cuadrante : `mentality_penalties`, `attacking_finishing`, `movement_acceleration` son  habilidades de atacantes, lo cual significa que los jugadores ofensivos *top* aparecerán aquí.
- 3º cuadrante  `mentality_vision`, `power_long_shots`, `skill_ball_control) aparecen habilidades de mediocampistas, lo cual significa que los jugadores del medio campo *top* aparecerán aquí.
- 4º cuadrante:   `defending_sliding_tackle`, `mentality_aggression`, `mentality_interceptions`,( habilidades de defensores) lo cual significa que los jugadores defensivos *top* aparecerán aquí.

Todas estas observaciones se evidenciarán en el siguiente gráfico.

**Graficamos a los jugadores usando las dos primeras componentes de PCA**

Proyectamos los datos (de **dim 28**) en ese nuevo espacio (de **dim 2**).

Identificamos a un jugador en particular, por ejemplo, Lionel Messi.

In [30]:
# Elegimos a nuestro jugador.
bool_PLAYER = pcaDF[name] == 'L. Messi'

# Diferenciamos a los mejores jugadores.
bool_TOP = pcaDF[overall] > topOverall

In [31]:
top = go.Scatter(x=X_projected[bool_TOP, 0],
                 y=X_projected[bool_TOP, 1],
                 name='Top Players',
                 text=pcaDF.loc[bool_TOP, name],
                 textfont=dict(family='sans serif', size=8, color='black'),
                 opacity=0.9,
                 marker=dict(color=palette[0], size=6),
                 mode='markers+text'
                )

average = go.Scatter(x=X_projected[~bool_TOP, 0],
                     y=X_projected[~bool_TOP, 1],
                     name='Average Players',
                     text=pcaDF.loc[~bool_TOP, name],
                     opacity=0.6,
                     marker=dict(color=palette[1], size=4),
                     mode='markers'
                    )

player = go.Scatter(x=X_projected[bool_PLAYER, 0],
                    y=X_projected[bool_PLAYER, 1],
                    name='Searched Player',
                    text=pcaDF.loc[bool_PLAYER, name],
                    textfont=dict(family='sans serif', size=10, color='black'),
                    opacity=1,
                    marker=dict(color=palette[4], size=20),
                    mode='markers+text'
                   )

data = [average, top, player]

layout = go.Layout(title='PCA - FIFA Players',
                   titlefont=dict(size=20),
                   xaxis=dict(title='Componente 0'),
                   yaxis=dict(title='Componente 1'),
                   autosize=False,
                   width=900,
                   height=900
                  )

fig = go.Figure(data=data, layout=layout)
fig.show()

ANALISIS

Un jugador *top* posee un `overall` alto.
Por lo tanto, tendrá valores altos en sus *skills* y estará ubicado en el plano de forma proporcional a los pesos de los mismos con respecto a las componentes de **PCA**.

En sentido antihorario...

- En el 1º cuadrante no hay jugadores *top* como esperábamos.
- En el 2º cuadrante esperábamos tener atacantes *top*. Algunos aparecen en el 3º cuadrante, como *Aubameyang*,  
- En el 3º cuadrante tenemos mediocampistas *top*, como *De Bruyne*,  *Kroos*. Aunque también se filtraron delanteros, como *Ronaldo*, . Otro testimonio sobre la ambivalencia de los jugadores.
- En el 4º cuadrante tenemos defensores *top*, como *Van Dijk*, *Piqué*, *Chiellini*.

**Graficamos a los jugadores usando las dos primeras componentes de PCA**

Proyectamos los datos (de **dim 28**) en ese nuevo espacio (de **dim 2**).

Ahora también identificamos su posición en el campo de juego.

In [32]:
bool_D = pcaDF[position_type] == 'Defender'

bool_M = pcaDF[position_type] == 'Midfielder'

bool_A = pcaDF[position_type] == 'Attacker'

In [33]:
attacker = go.Scatter(x=X_projected[bool_A, 0],
                      y=X_projected[bool_A, 1],
                      name='Attackers',
                      text=pcaDF.loc[bool_A, name],
                      opacity=0.8,
                      marker=dict(color=palette[0], size=5),
                      mode='markers'
                     )

midfielder = go.Scatter(x=X_projected[bool_M, 0],
                        y=X_projected[bool_M, 1],
                        name='Midfielders',
                        text=pcaDF.loc[bool_M, name],
                        opacity=0.7,
                        marker=dict(color=palette[1], size=5),
                        mode='markers'
                       )

defender = go.Scatter(x=X_projected[bool_D, 0],
                      y=X_projected[bool_D, 1],
                      name='Defenders',
                      text=pcaDF.loc[bool_D, name],
                      opacity=0.6,
                      marker=dict(color=palette[2], size=5),
                      mode='markers'
                     )

top = go.Scatter(x=X_projected[bool_TOP, 0],
                 y=X_projected[bool_TOP, 1],
                 name='Top Players',
                 text=pcaDF.loc[bool_TOP, name],
                 textfont=dict(family='sans serif', color='black', size=10),
                 opacity=0.9,
                 mode='text'
                )

data = [attacker, midfielder, defender, top]

layout = go.Layout(title='PCA - FIFA Players',
                   titlefont=dict(size=20),
                   xaxis=dict(title='Componente 0'),
                   yaxis=dict(title='Componente 1'),
                   autosize=False,
                   width=900,
                   height=900
                  )

fig = go.Figure(data=data, layout=layout)
fig.show()

# 6.Clustering_PCA



agrupamientos de los jugadores usando las primeras componentes.



**KMeans**

Intentamos agrupar sobre las primeras componentes de *PCA*.
Se define de forma arbitraria la cantidad de conjuntos a buscar.

In [36]:
# Número de clusters buscados.
n_clust = 3

km = KMeans(n_clusters=n_clust, random_state=123)
km.fit(X_projected) # Utiliza todas las componentes: 5 dimensiones.

# Recuperación de las etiquetas.
clusters = km.labels_

print(f'Suma de los cuadrados de las distancias al cluster (Inertia) {km.inertia_}')

Suma de los cuadrados de las distancias al cluster (Inertia) 26573.509962059623


In [37]:
clustersDF = pcaDF.copy()
clustersDF[cluster_KMeans] = clusters.astype(str) # Clusters como Variables Categóricas
clustersDF[pca_0] = X_projected[:, 0]
clustersDF[pca_1] = X_projected[:, 1]

print(f'KMeans encontró {max(clusters) + 1} clusters (nosotros forzamos la cantidad)')

KMeans encontró 3 clusters (nosotros forzamos la cantidad)


In [38]:
fig = px.scatter(clustersDF,
                 x=pca_0,
                 y=pca_1,
                 color=cluster_KMeans,
                 hover_name=name,
                 hover_data=description,
                 symbol=player_class,
                 symbol_map={'TOP': 'star', 'Average': 'circle'},
                 title='Clustering KMeans',
                 height=800,
                 width=800)

fig.show()

**Observación**

 **KMeans** separa los tres conjuntos de jugadores de campo de forma mas o menos satisfactoria.
se obtuvo un agrupamiento mucho mejor cuando se aplicó *clustering* sobre el resultado del *embedding* de *tSNE*.



**MeanShift**

Intentamos agrupar sobre los componentes de *PCA*.
Al estimar el ancho de banda óptimo (`4.85`) para aplicar el agrupamiento, solo se encontró **1** conjunto.
Debido que esto no es el resultado que se buscaba, se definió un valor arbitrario para efectuar el *clustering*.

In [39]:
# Estimate BW for MeanShift.
estimated_BW = estimate_bandwidth(X_projected, random_state=123)
print(f'BandWidth Estimado: {estimated_BW}')

# Utilizamos el ancho de banda encontrado.
ms = MeanShift(bandwidth=estimated_BW, bin_seeding=True)
ms.fit(X_projected) # Utiliza todas las componentes: 5 dimensiones.

# Recuperación de las etiquetas.
clusters = ms.labels_

print(f'MeanShift encontró {max(clusters) + 1} clusters (según los hiperparámetros elegidos)')

BandWidth Estimado: 4.8500079646854095
MeanShift encontró 1 clusters (según los hiperparámetros elegidos)


In [40]:
# Utilizamos un ancho de banda arbitrario.
ms = MeanShift(bandwidth=2.75, bin_seeding=True)
ms.fit(X_projected) # Utiliza todas las componentes: 5 dimensiones.

# Recuperación de las etiquetas.
clusters = ms.labels_

In [41]:
clustersDF[cluster_MeanShift] = clusters.astype(str) # Clusters como Variables Categóricas

print(f'MeanShift encontró {max(clusters) + 1} clusters (según los hiperparámetros elegidos)')

MeanShift encontró 3 clusters (según los hiperparámetros elegidos)


In [42]:
fig = px.scatter(clustersDF,
                 x=pca_0,
                 y=pca_1,
                 color=cluster_MeanShift,
                 hover_name=name,
                 hover_data=description,
                 symbol=player_class,
                 symbol_map={'TOP': 'star', 'Average': 'circle'},
                 title='Clustering MeanShift',
                 height=800,
                 width=800)

fig.show()

ANALISIS


conjunto de datos es SEPARADO en dos grandes grupos  donde hay jugadores ofensivos de un lado y jugadores defensivos del otro. Peor aparece un TERCER GRUPO (verde) que no se puede explicar muy bien (es raro). Son adatos dispersos sin aparente  relación

